In [ ]:
# !pip install numpy
import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

In [ ]:
X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

In [ ]:
X.shape , W.shape , b.shape , num_movies , num_features , num_users

In [ ]:
Y.shape , R.shape

In [ ]:
movie_list  , df = load_Movie_List_pd()

In [ ]:
def cost_function_for_collabfiltering(X,W,B,Y,R,lambda_):
  j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
  J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
  return J

In [ ]:
cost_function_for_collabfiltering(X,W,b,Y,R,1)

In [ ]:
personalised_rating = np.zeros((num_movies,1))
personalised_rating[62] = 1
personalised_rating[2716] = 5
personalised_rating[3618] = 1
personalised_rating[1398] = 5
personalised_rating[3083] = 5
personalised_rating[2112] = 3
personalised_rating[2755] = 4
personalised_rating[4170] = 4
personalised_rating[378] = 5
personalised_rating[3773] = 3
personalised_rating[4415] = 4
personalised_rating[3009] = 1
personalised_rating[1088] = 1
personalised_rating[3556] = 5

my_rated = [i for i in range(num_movies) if personalised_rating[i] > 0  ]
for i in range(num_movies):
  if(personalised_rating[i] > 0):
    print(f"User rated {float(personalised_rating[i])} to movie : {movie_list[i]}")

In [ ]:
#This adds/concatenate along the second axis
Y = np.c_[personalised_rating , Y]
R = np.c_[(personalised_rating != 0).astype(int) , R]

In [ ]:
normalised_y , mean_y = normalizeRatings(Y,R)

In [ ]:
num_movies, num_users = Y.shape
num_features = 100
# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [ ]:
iterations = 300
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cost_function_for_collabfiltering(X,W,b,normalised_y,R,1)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

In [ ]:
predictions_normalised =  np.matmul(X.numpy(),W.numpy().T) + b.numpy()
prediction = predictions_normalised + mean_y
pred = prediction[:,0]
print(pred)
# This give the index of current value in the sorted vector , so descending sort of this array can give use the index of the largest value in the current vector
ix = tf.argsort(pred, direction='DESCENDING')


In [ ]:
print("The Below given movie list is the most recommended according to the Collaborative filtering")
for i in range(20):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {pred[j]:0.2f} for movie {movie_list[j]}')

In [ ]:
for i in range(len(personalised_rating)):
    if personalised_rating[i] > 0:
        print(f'Original {personalised_rating[i]}, Predicted {pred[i]:0.2f} for {movie_list[i]}')